In [30]:
import pandas as pd
import subprocess
import os
import ntpath

import requests
from bs4 import BeautifulSoup

import json

pd.set_option('display.max_rows', 500)

# Data Understanding

## git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [33]:
git_pull = subprocess.Popen( "/usr/bin/git pull",
                     cwd = os.path.dirname('C:\\Users\\SAYALI\\ads_covid-19\\' ),
                     shell = True,
                     stdout = subprocess.PIPE,
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()

print("Error : " + str(error))
print("out : " + str(out))

Error : b'The system cannot find the path specified.\r\n'
out : b''


In [36]:
data_path='/users/sayali/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)


In [37]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34740,34994,35070,35229,35301,35475,35526,35615,35727,35928
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3667,3752,3851,3906,4008,4090,4171,4290,4358,4466
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,20216,20770,21355,21948,22549,23084,23691,24278,24872,25484
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,861,862,877,880,880,880,884,884,889,889
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,541,576,607,638,687,705,749,779,812,851


## Webscrapping

In [38]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [39]:
soup = BeautifulSoup(page.content, 'html.parser')

In [41]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [42]:
all_rows=html_table.find_all('tr')

In [43]:
final_data_list=[]

In [44]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)

In [50]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [51]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,36.731,+166,401,"3,6",1.839
3,Bayern,50.112,+63,442,"3,4",2.619
4,Berlin,8.894,+34,148,"3,9",222
5,Branden­burg,3.523,+13,50,"2,0",168
6,Bremen,1.733,+6,32,"4,7",55


## Rest API Calls

In [52]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [53]:
json_object=json.loads(data.content)

In [54]:
type(json_object)

dict

In [55]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [56]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [57]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,3299,1595541600000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,113.887746,4.573731e+10,2.881496e+06,155
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,5275,1595541600000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,286.501204,2.089396e+09,4.188002e+05,261
2,3,03,Niedersachsen,Land,7982448,9,14185,1595541600000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,177.702379,1.299836e+11,4.008988e+06,647
3,4,04,Bremen,Freie Hansestadt,682986,5,1733,1595541600000,04,4132268b-54de-4327-ac1e-760e915112f1,253.738730,1.119157e+09,3.357177e+05,55
4,5,05,Nordrhein-Westfalen,Land,17932651,10,46956,1595541600000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,261.846394,8.782936e+10,2.648673e+06,1726


In [60]:
pd_full_list.to_csv('/users/sayali/ads_covid-19/data/raw/NPGEO/GER_state_data.csv',sep=';')

In [61]:
pd_full_list.shape[0]

16

## API access via REST service, e.g. USA data

In [67]:
import requests

In [69]:
url_endpoint = 'https://api.smartable.ai/coronavirus/stats/US'

headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key' : 'd17ab1dc03dc44b698b1d1c3d0201900',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

<Response [200]>


In [71]:
US_dict=json.loads(response.content) # imports string
with open('/users/sayali/ads_covid-19/data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

In [72]:
print(json.dumps(US_dict,indent=2)) #string dump

{
  "location": {
    "long": -95.712891,
    "countryOrRegion": "United States",
    "provinceOrState": null,
    "county": null,
    "isoCode": "US",
    "lat": 37.09024
  },
  "updatedDateTime": "2020-07-24T16:01:32.9550587Z",
  "stats": {
    "totalConfirmedCases": 4127752,
    "newlyConfirmedCases": 20901,
    "totalDeaths": 147423,
    "newDeaths": 293,
    "totalRecoveredCases": 1931845,
    "newlyRecoveredCases": 341,
    "history": [
      {
        "date": "2020-01-22T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-23T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-24T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-25T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-26T00:00:00",
        "c

In [73]:
US_dict['stats']['breakdowns'][0]

{'location': {'long': 144.793731,
  'countryOrRegion': 'United States',
  'provinceOrState': 'Guam',
  'county': None,
  'isoCode': None,
  'lat': 13.444304},
 'totalConfirmedCases': 32,
 'newlyConfirmedCases': 0,
 'totalDeaths': 1,
 'newDeaths': 0,
 'totalRecoveredCases': 0,
 'newlyRecoveredCases': 0}

In [74]:
full_list_US_country=[]
for pos,each_dict in enumerate (US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_US_country.append(flatten_dict)

In [76]:
pd.DataFrame(full_list_US_country).to_csv('/users/sayali/ads_covid-19/data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)
